In [1]:
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

/home/jianwei2/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jianwei2/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jianwei2/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jianwei2/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

In [7]:
# data path
DATASET_PATH  = 'dataset'

# image size
IMAGE_SIZE = (224, 224)

# class number
NUM_CLASSES = 12

# if lacking GPU memory， you can lower the batch size or reduce the freeze layers
BATCH_SIZE = 8

# freeze layers
FREEZE_LAYERS = 2

# Epoch number
NUM_EPOCHS = 20

# trained model name
WEIGHTS_FINAL = 'model-resnet50-final.h5'

In [8]:
# 透過 data augmentation 產生訓練與驗證用的影像資料
train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

In [9]:
train_batches = train_datagen.flow_from_directory(DATASET_PATH + '/train',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)

Found 8462 images belonging to 12 classes.


In [10]:
valid_datagen = ImageDataGenerator()
valid_batches = valid_datagen.flow_from_directory(DATASET_PATH + '/test',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=BATCH_SIZE)

Found 2124 images belonging to 12 classes.


In [11]:
# index of output class
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))

# 以訓練好的 ResNet50 為基礎來建立模型，
# 捨棄 ResNet50 頂層的 fully connected layers
net = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
x = net.output
x = Flatten()(x)

# 增加 DropOut layer
x = Dropout(0.5)(x)

# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)

# 設定凍結與要進行訓練的網路層
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True

# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])

# 輸出整個網路結構
print(net_final.summary())

# 訓練模型
net_final.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS)

# 儲存訓練好的模型
net_final.save(WEIGHTS_FINAL)

Class #0 = C
Class #1 = CB
Class #2 = DE
Class #3 = DT
Class #4 = LB
Class #5 = OG
Class #6 = OT
Class #7 = QB
Class #8 = RB
Class #9 = S
Class #10 = TE
Class #11 = WR
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormaliz

Epoch 1/20
1057/1057 [==============================] - 3959s 4s/step - loss: 3.5598 - acc: 0.1695 - val_loss: 2.1837 - val_acc: 0.3165
Epoch 2/20
1057/1057 [==============================] - 4548s 4s/step - loss: 2.3246 - acc: 0.2535 - val_loss: 1.4253 - val_acc: 0.4392
Epoch 3/20
1057/1057 [==============================] - 4000s 4s/step - loss: 1.6810 - acc: 0.4028 - val_loss: 0.9803 - val_acc: 0.6108
Epoch 4/20
1057/1057 [==============================] - 3944s 4s/step - loss: 1.3198 - acc: 0.5106 - val_loss: 0.9375 - val_acc: 0.6255
Epoch 5/20
1057/1057 [==============================] - 3931s 4s/step - loss: 1.1356 - acc: 0.5727 - val_loss: 0.7738 - val_acc: 0.6726
Epoch 6/20
1057/1057 [==============================] - 3927s 4s/step - loss: 0.9966 - acc: 0.6228 - val_loss: 0.7595 - val_acc: 0.6830
Epoch 7/20
1057/1057 [==============================] - 3927s 4s/step - loss: 0.8998 - acc: 0.6505 - val_loss: 0.6486 - val_acc: 0.7406
Epoch 8/20
1057/1057 [==========================